In [98]:
import os 
import sys
import random
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.stats import lognorm
from scipy.stats import norm 
from scipy import integrate
from scipy.stats import gmean

from scipy.stats import pearsonr

import statsmodels.api as sm 
import statsmodels.formula.api as smf

### src; https://github.com/arkottke/pyrotd
import pyrotd
pd.set_option('display.max_rows', 100)
from scipy import stats

from efficiency_OLS import OLS
from Linear_model_fits import fit_against_IM_and_M, fit_against_IM_and_M_Rjb, fit_against_IM_and_Rjb
from Linear_model_fits import fit_IM_against_M, fit_IM_against_Rjb, fit_IM_against_M_and_Rjb
from extract_edp_data import extract_EDP_data
from sufficiency_kl_divergence import compute_KL_divergence, compile_kl_divergence, summarize_agg_kld
from sufficiency_kl_divergence import compile_kld_mvn_EDPs, summarize_agg_kld_mvn
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
with open('BuildingNames.txt', 'r') as f:
    BuildingList = f.read() 
BuildingList = BuildingList.split('\n')

baseDir = r'/Users/laxmandahal/Desktop/UCLA/Phd/Research/IM_study'

## time period of the buildings
# T = np.array([0.13, 0.12, 0.22, 0.22, 0.16, 0.15, 0.26, 0.25, 0.49, 0.49])
T = np.array([0.13, 0.12, 0.16, 0.15, 0.22, 0.22, 0.26, 0.25, 0.49, 0.49])

numGM = 826
g = 980.665 ## converts GM record to cm/sec^2

def compute_RotDxx_EDP(edpX, edpZ, percentile = 50):
    angles = np.arange(0, 180, step=1)
    radians = np.radians(angles)
    coeffs = np.c_[np.cos(radians), np.sin(radians)]
    
    edp_stacked = np.vstack([edpX, edpZ])
    rotated_edp = np.dot(coeffs, edp_stacked)
    percentile_edp = np.percentile(rotated_edp, q = percentile, axis = 0, interpolation='linear')
    return percentile_edp


In [77]:
BuildingList

['s1_48x32_high',
 's1_48x32_veryhigh',
 's1_96x48_high',
 's1_96x48_veryhigh',
 's2_48x32_high',
 's2_48x32_veryhigh',
 's2_96x48_high',
 's2_96x48_veryhigh',
 's4_96x48_high',
 's4_96x48_veryhigh']

In [3]:
gminfo_rotD50 = pd.read_csv(r'data/Complete_GM_info_RotD50.csv')
gminfo_rotD50 = gminfo_rotD50.set_index('key_0')
gminfo_rotD50.index.names = ['GMID']

In [27]:
buildingIndex = 0
pairingID = 1
IMs = 'SaT1'
numGM = 826

if pairingID == 1:
    start_index_multiplier = 0
    end_index_multiplier = 1
else:
    start_index_multiplier = 2
    end_index_multiplier = 3

if IMs == 'SaT1':
    IM = gminfo_rotD50['T_%s'%T[buildingIndex]].values
else:
    IM = gminfo_rotD50[IMs]
    
baseDir = r'/Users/laxmandahal/Desktop/UCLA/Phd/Research/IM_study'
dataDir = os.path.join(baseDir, *['Results', 'IM_study_826GMs', BuildingList[buildingIndex]])

os.chdir(dataDir)
sdr = pd.read_csv(os.path.join(dataDir, 'SDR.csv'), header = None)
pfa = pd.read_csv(os.path.join(dataDir, 'PFA.csv'), header = None)

#story index
i = 0
j = 0
sdrX = sdr[3+i].values[numGM * start_index_multiplier : numGM * end_index_multiplier]
sdrZ = sdr[3+i].values[numGM * end_index_multiplier : numGM * (end_index_multiplier + 1)]

pfaX = pfa[4+i].values[numGM * start_index_multiplier : numGM * end_index_multiplier]
pfaZ = pfa[4+i].values[numGM * end_index_multiplier : numGM * (end_index_multiplier + 1)]

sdr_rotd50 = compute_RotDxx_EDP(sdrX, sdrZ, percentile=50)
pfa_rotd50 = compute_RotDxx_EDP(pfaX, pfaZ, percentile=50)
X = sm.add_constant(np.log(IM))

sdrX_avg = gmean([sdr[3+j].values[0:numGM], sdr[3+j].values[numGM*2:numGM*3]])
pfaX_avg = gmean([pfa[3+j].values[0:numGM], pfa[3+j].values[numGM*2:numGM*3]])

sdrZ_avg = gmean([sdr[3+j].values[numGM:numGM*2], sdr[3+j].values[numGM*3:numGM*4]])
pfaZ_avg = gmean([pfa[3+j].values[numGM:numGM*2], pfa[3+j].values[numGM*3:numGM*4]])
# print(pfaZ_avg.shape)
#                 compute_RotDxx_EDP(sdrX, sdrZ, percentile=50)
#                 pfa_rotD50 = compute_RotDxx_EDP(pfaX, pfaZ, percentile=50)
ols_sdrX = OLS(sdrX, IM)
ols_sdrZ = OLS(sdrZ, IM)
ols_pfaX = OLS(pfaX, IM)
ols_pfaZ = OLS(pfaZ, IM)

ols_sdrX_M = fit_against_IM_and_M(sdrX, IM, gminfo_rotD50['Magnitude'].values)

In [28]:
print(stats.shapiro(ols_sdrX.result.get_prediction().predicted_mean),
      stats.shapiro(ols_sdrZ.result.get_prediction().predicted_mean),
     stats.shapiro(ols_pfaX.result.get_prediction().predicted_mean),
     stats.shapiro(ols_pfaZ.result.get_prediction().predicted_mean))

ShapiroResult(statistic=0.9929214119911194, pvalue=0.0005802168161608279) ShapiroResult(statistic=0.9929214119911194, pvalue=0.0005802168161608279) ShapiroResult(statistic=0.9929206967353821, pvalue=0.0005797366029582918) ShapiroResult(statistic=0.9929208755493164, pvalue=0.0005798561032861471)


In [85]:
def kl_divergence_mvn(m0, S0, m1, S1):
    """
    Kullback-Liebler divergence from Gaussian pm,pv to Gaussian qm,qv.
    Also computes KL divergence from a single Gaussian pm,pv to a set
    of Gaussians qm,qv.
    

    From wikipedia
    KL( (m0, S0) || (m1, S1))
         = .5 * ( tr(S1^{-1} S0) + log |S1|/|S0| + 
                  (m1 - m0)^T S1^{-1} (m1 - m0) - N )
    """
    
    # store inv diag covariance of S1 and diff between means
    N = m0.shape[0]
    iS1 = np.linalg.inv(S1)
    diff = m1 - m0

    # kl is made of three terms
    tr_term   = np.trace(iS1 @ S0)
    det_term  = np.log(np.linalg.det(S1)/np.linalg.det(S0)) #np.sum(np.log(S1)) - np.sum(np.log(S0))
    quad_term = diff.T @ np.linalg.inv(S1) @ diff #np.sum( (diff*diff) * iS1, axis=1)
    #print(tr_term,det_term,quad_term)
    return .5 * (tr_term + det_term + quad_term - N) 

In [93]:
mu1 = np.array([np.mean(ols_sdrX.result.get_prediction().predicted_mean),
                np.mean(ols_pfaX.result.get_prediction().predicted_mean)])
cov1 = np.cov(ols_sdrX.result.get_prediction().predicted_mean,
              ols_pfaX.result.get_prediction().predicted_mean, ddof=0)
mu2 = np.array([np.mean(ols_sdrZ.result.get_prediction().predicted_mean),
                np.mean(ols_pfaZ.result.get_prediction().predicted_mean)])
cov2 = np.cov(ols_sdrZ.result.get_prediction().predicted_mean,
              ols_pfaZ.result.get_prediction().predicted_mean, ddof=0)
kl_divergence_mvn(mu1, cov1, mu2, cov2)

4028085329695.222

In [96]:
from scipy.special import rel_entr
sum(rel_entr(ols_sdrX.result.get_prediction().predicted_mean,ols_pfaX.result.get_prediction().predicted_mean))

inf

In [94]:
np.cov(ols_sdrX.result.get_prediction().predicted_mean,
              ols_pfaX.result.get_prediction().predicted_mean)

array([[1.48809565, 1.39864155],
       [1.39864155, 1.31456482]])

In [111]:
comDat = np.array([ols_sdrX.result.get_prediction().predicted_mean,
                 ols_sdrZ.result.get_prediction().predicted_mean])
np.cov(comDat, ddof=1)

array([[1.48809565, 1.47567175],
       [1.47567175, 1.46335158]])

In [112]:
np.linalg.det(np.cov(comDat, ddof=0))

0.0

In [29]:
stats.shapiro(ols_sdrX_M.get_prediction().predicted_mean)

ShapiroResult(statistic=0.9928898811340332, pvalue=0.0005583394085988402)

In [30]:
stats.shapiro(np.log(IM))

ShapiroResult(statistic=0.9929210543632507, pvalue=0.0005799776408821344)

In [21]:
stats.shapiro(np.log(gminfo_rotD50['Magnitude']))

ShapiroResult(statistic=0.9372308254241943, pvalue=3.642206518017458e-18)

In [47]:
stats.shapiro(np.log(gminfo_rotD50['Distance_Rjb']))

ShapiroResult(statistic=0.9448215365409851, pvalue=5.490531091573419e-17)

In [41]:
stats.shapiro(np.log(gminfo_rotD50['T_0.26']))

ShapiroResult(statistic=0.9965856671333313, pvalue=0.07254207879304886)

In [45]:
stats.shapiro(np.log(gminfo_rotD50['SI']))

ShapiroResult(statistic=0.9939128756523132, pvalue=0.0020030317828059196)

In [26]:
ols_IM = fit_IM_against_M(gminfo_rotD50['T_0.23'],gminfo_rotD50['Magnitude'] )
stats.shapiro(ols_IM.get_prediction().predicted_mean)

ShapiroResult(statistic=0.9372313022613525, pvalue=3.642842619908511e-18)

In [78]:
IM_list = ['SaT1', 'PGA', 'PGV', 'Sa_avg', 'CAV', 'SI', 'ASI', 'DSI','DS_5to75', 'DS_5to95' ]

b = compile_kl_divergence(BuildingList, 9, baseDir, IM_list, gminfo_rotD50, pairingID=1, numGM=826,
                          remove_collapse= False, KL_on_residual=True, use_predicted_IM=False)
b

IM,SaT1,PGA,PGV,Sa_avg,CAV,SI,ASI,DSI,DS_5to75,DS_5to95
"KL(sdrX|im, sdrX|im,M)",0.000240,8.957884e-04,0.000003,2.010379e-04,0.000154,1.274097e-09,1.454360e-03,3.767535e-09,0.008935,0.005766
"KL(sdrZ|im, sdrZ|im,M)",0.000352,1.378655e-03,0.000010,2.891918e-04,0.000406,1.776515e-08,2.154831e-03,1.257434e-08,0.012157,0.007187
"KL(pfaX|im, pfaX|im,M)",0.000245,1.434685e-08,0.002047,4.493098e-04,0.012305,1.006796e-03,3.451064e-05,1.927541e-04,0.006698,0.001838
"KL(pfaZ|im, pfaZ|im,M)",0.000748,1.109524e-05,0.003116,1.827201e-03,0.018499,1.390711e-03,7.155687e-10,2.771981e-04,0.006050,0.001561
"KL(sdrX|im, sdrX|im,R)",0.003053,2.695342e-03,0.007020,1.423155e-03,0.027518,4.376955e-03,3.527109e-03,3.914693e-02,0.073360,0.071446
"KL(sdrZ|im, sdrZ|im,R)",0.001262,8.201756e-04,0.003867,3.723781e-04,0.022447,2.561740e-03,1.487946e-03,3.358103e-02,0.058403,0.058421
"KL(pfaX|im, pfaX|im,R)",0.000007,7.929486e-06,0.003221,1.364389e-05,0.016683,1.909133e-03,2.114694e-05,3.227860e-02,0.030209,0.033993
"KL(pfaZ|im, pfaZ|im,R)",0.000218,5.376725e-06,0.005751,1.779006e-09,0.023621,4.332000e-03,1.561278e-06,4.478385e-02,0.036804,0.039317
"KL(sdrX|im, sdrX|im,M,R)",0.010703,1.413637e-02,0.009416,6.202251e-03,0.029781,6.533461e-03,1.907809e-02,5.830814e-02,0.178103,0.163209
"KL(sdrZ|im, sdrZ|im,M,R)",0.006559,8.922188e-03,0.004517,3.110932e-03,0.022719,3.604435e-03,1.422889e-02,4.885019e-02,0.169284,0.149857


In [79]:
b.iloc[:8].sum()

IM
SaT1        0.006125
PGA         0.005814
PGV         0.025034
Sa_avg      0.004576
CAV         0.121632
SI          0.015577
ASI         0.008681
DSI         0.150260
DS_5to75    0.232617
DS_5to95    0.219529
dtype: float64

In [80]:
b.iloc[8:].sum()

IM
SaT1        0.018647
PGA         0.023123
PGV         0.027660
Sa_avg      0.012711
CAV         0.129800
SI          0.018873
ASI         0.033384
DSI         0.185669
DS_5to75    0.510820
DS_5to95    0.437911
dtype: float64

In [83]:
from sufficiency_kl_divergence import summarize_agg_kld

sumdf = summarize_agg_kld(BuildingList, baseDir, IM_list, gminfo_rotD50, pairingID=1, numGM=826,
                          remove_collapse= False, KL_on_residual=False, use_predicted_IM=False,
                         mv_causal_params=False)
# sumdf
sumdf.style.highlight_min(color = 'lightgreen', axis = 1)

IM,SaT1,PGA,PGV,Sa_avg,CAV,SI,ASI,DSI,DS_5to75,DS_5to95
s1_48x32_high,0.000003,0.000062,0.031032,0.000555,0.067178,0.021163,0.000024,0.352207,0.120632,0.196247
s1_48x32_veryhigh,0.000002,0.000055,0.030117,0.000516,0.065806,0.020099,0.000023,0.338933,0.123624,0.198992
s1_96x48_high,0.000063,0.000124,0.038967,0.000851,0.079341,0.028530,0.000094,0.396621,0.126498,0.203058
s1_96x48_veryhigh,0.000151,0.000194,0.041929,0.001081,0.085242,0.031388,0.000192,0.408302,0.135437,0.212775
s2_48x32_high,0.000139,0.000124,0.034536,0.000689,0.074284,0.025154,0.000111,0.373706,0.131358,0.208743
s2_48x32_veryhigh,0.001089,0.000743,0.042015,0.001676,0.089355,0.032499,0.000846,0.402181,0.170217,0.241421
s2_96x48_high,0.000860,0.000465,0.026249,0.000726,0.068255,0.017748,0.000550,0.305190,0.167622,0.239585
s2_96x48_veryhigh,0.000627,0.000357,0.017972,0.000386,0.047402,0.012405,0.000350,0.254391,0.236422,0.295590
s4_96x48_high,0.002402,0.002848,0.009478,0.001239,0.045851,0.004960,0.004584,0.170310,0.344391,0.358112
s4_96x48_veryhigh,0.000708,0.001460,0.007088,0.000356,0.027741,0.003467,0.002098,0.156098,0.432185,0.428226


In [84]:
sumdf = summarize_agg_kld(BuildingList, baseDir, IM_list, gminfo_rotD50, pairingID=1, numGM=826,
                          remove_collapse= False, KL_on_residual=False, use_predicted_IM=False,
                         mv_causal_params=True)
# sumdf
sumdf.style.highlight_min(color = 'lightgreen', axis = 1)

IM,SaT1,PGA,PGV,Sa_avg,CAV,SI,ASI,DSI,DS_5to75,DS_5to95
s1_48x32_high,0.000003,0.000064,0.027026,0.000560,0.057947,0.019651,0.000030,0.334300,0.141655,0.213905
s1_48x32_veryhigh,0.000002,0.000058,0.026252,0.000523,0.056807,0.018680,0.000029,0.322000,0.145346,0.217471
s1_96x48_high,0.000064,0.000150,0.033806,0.000917,0.067990,0.026318,0.000118,0.375764,0.145945,0.218807
s1_96x48_veryhigh,0.000151,0.000239,0.036347,0.001185,0.072869,0.028904,0.000230,0.386601,0.154470,0.228026
s2_48x32_high,0.000140,0.000153,0.030143,0.000761,0.064001,0.023300,0.000125,0.357164,0.153144,0.227318
s2_48x32_veryhigh,0.001104,0.000768,0.036889,0.001761,0.077113,0.030148,0.000851,0.386357,0.191996,0.259976
s2_96x48_high,0.000898,0.000467,0.023271,0.000760,0.059670,0.016581,0.000565,0.297943,0.196972,0.267572
s2_96x48_veryhigh,0.001093,0.000535,0.016289,0.000405,0.042192,0.011889,0.000613,0.255516,0.273951,0.335201
s4_96x48_high,0.003527,0.005366,0.009014,0.001914,0.043483,0.004952,0.008520,0.176944,0.386557,0.410205
s4_96x48_veryhigh,0.002015,0.004494,0.007354,0.001193,0.026410,0.003901,0.006029,0.170395,0.466896,0.481000


In [97]:
from sufficiency_kl_divergence import compile_kld_mvn_EDPs
b = compile_kld_mvn_EDPs(BuildingList, 9, baseDir, IM_list, gminfo_rotD50, pairingID=1, numGM=826,
                          remove_collapse= False, KL_on_residual=True, use_predicted_IM=False)
b

IM,SaT1,PGA,PGV,Sa_avg,CAV,SI,ASI,DSI,DS_5to75,DS_5to95
"KL(sdr+pfaX|im, sdr+pfaX|im,M)",0.000929,0.000930,0.002419,0.001129,0.012867,0.001310,0.001680,0.000523,0.010953,0.005822
"KL(sdr+pfaZ|im, sdr+pfaZ|im,M)",0.001956,0.001730,0.003629,0.002867,0.018882,0.001990,0.002162,0.000924,0.012922,0.007256
"KL(sdr+pfaX|im, sdr+pfaX|im,R)",0.003536,0.003983,0.010360,0.001609,0.046961,0.006984,0.005100,0.058211,0.079641,0.081129
"KL(sdr+pfaZ|im, sdr+pfaZ|im,R)",0.003107,0.001005,0.009321,0.000408,0.048215,0.006973,0.001581,0.059496,0.069300,0.071295
"KL(sdr+pfaX|im, sdr+pfaX|im,M,R)",0.011100,0.017889,0.012452,0.011495,0.053240,0.008547,0.020141,0.072134,0.206564,0.180889
"KL(sdr+pfaZ|im, sdr+pfaZ|im,M,R)",0.007691,0.011290,0.011075,0.006843,0.062035,0.008028,0.014346,0.068732,0.199707,0.165917


In [100]:
sumdf_mvn = summarize_agg_kld_mvn(BuildingList, baseDir, IM_list, gminfo_rotD50, pairingID=1, numGM=826,
                          remove_collapse= False, KL_on_residual=True, use_predicted_IM=False,
                         mv_causal_params=False)
# sumdf
sumdf_mvn.style.highlight_min(color = 'lightgreen', axis = 1)

IM,SaT1,PGA,PGV,Sa_avg,CAV,SI,ASI,DSI,DS_5to75,DS_5to95
s1_48x32_high,0.006346,0.010936,0.061957,0.024773,0.182598,0.049805,0.008138,0.179278,0.141351,0.144865
s1_48x32_veryhigh,0.004152,0.009748,0.057653,0.023038,0.181459,0.045315,0.006545,0.169384,0.140809,0.144284
s1_96x48_high,0.008695,0.010320,0.066118,0.025280,0.185761,0.054612,0.009883,0.192886,0.147245,0.151100
s1_96x48_veryhigh,0.012432,0.012771,0.072964,0.028931,0.200635,0.060757,0.012873,0.207533,0.163120,0.166709
s2_48x32_high,0.018373,0.013227,0.076753,0.027596,0.209110,0.062749,0.016720,0.226023,0.186199,0.189083
s2_48x32_veryhigh,0.026419,0.019784,0.087080,0.033398,0.232107,0.073155,0.023899,0.246506,0.216001,0.218581
s2_96x48_high,0.032195,0.017045,0.079766,0.026969,0.234230,0.063791,0.022289,0.239556,0.230590,0.229969
s2_96x48_veryhigh,0.012585,0.007388,0.043799,0.011503,0.167067,0.034168,0.008299,0.159924,0.179277,0.180365
s4_96x48_high,0.027817,0.017844,0.047067,0.013482,0.190591,0.035136,0.025834,0.177876,0.237773,0.228187
s4_96x48_veryhigh,0.009528,0.007648,0.025728,0.006013,0.126925,0.017257,0.010523,0.119155,0.172817,0.165503


In [101]:
sumdf_mvn = summarize_agg_kld_mvn(BuildingList, baseDir, IM_list, gminfo_rotD50, pairingID=1, numGM=826,
                          remove_collapse= False, KL_on_residual=True, use_predicted_IM=False,
                         mv_causal_params=True)
# sumdf
sumdf_mvn.style.highlight_min(color = 'lightgreen', axis = 1)

IM,SaT1,PGA,PGV,Sa_avg,CAV,SI,ASI,DSI,DS_5to75,DS_5to95
s1_48x32_high,0.006888,0.010766,0.061384,0.023866,0.211217,0.050297,0.009010,0.176835,0.186089,0.164998
s1_48x32_veryhigh,0.004781,0.009638,0.057316,0.022475,0.212204,0.045962,0.007459,0.167026,0.190791,0.167615
s1_96x48_high,0.008726,0.009912,0.063666,0.023573,0.206261,0.053481,0.009900,0.190514,0.192926,0.171470
s1_96x48_veryhigh,0.012433,0.012318,0.070564,0.026971,0.223139,0.059612,0.012917,0.204993,0.208819,0.187474
s2_48x32_high,0.018649,0.012708,0.072433,0.025438,0.216769,0.060838,0.016421,0.225840,0.258210,0.225657
s2_48x32_veryhigh,0.027172,0.019726,0.081567,0.031030,0.236859,0.070131,0.024154,0.246785,0.286972,0.258807
s2_96x48_high,0.033589,0.017635,0.074977,0.025027,0.234007,0.061401,0.023642,0.243337,0.360941,0.304545
s2_96x48_veryhigh,0.022709,0.015546,0.038905,0.014584,0.158395,0.031555,0.016928,0.169845,0.321724,0.280785
s4_96x48_high,0.036175,0.037093,0.043482,0.022150,0.176829,0.033466,0.050352,0.198823,0.499219,0.423206
s4_96x48_veryhigh,0.018791,0.029179,0.023526,0.018337,0.115275,0.016575,0.034487,0.140866,0.406271,0.346807


In [113]:
from suff_kld_prob_exceedance import compile_kld_prob_exceedance, summarize_kld_results

a = compile_kld_prob_exceedance(BuildingList, 9, baseDir, IM_list, gminfo_rotD50, pairingID=1, numGM=826,
                               remove_collapse=False, use_predicted_IM=False)
a

IM,SaT1,PGA,PGV,Sa_avg,CAV,SI,ASI,DSI,DS_5to75,DS_5to95
"KL(sdrX|im, sdrX|im,M)",1.643093e-06,6.746265e-06,2.100029e-08,1.369871e-06,0.000001,9.478085e-12,1.080608e-05,3.774270e-11,0.000093,0.000061
"KL(sdrZ|im, sdrZ|im,M)",1.944665e-06,8.572306e-06,7.030004e-08,1.596496e-06,0.000003,1.163883e-10,1.350962e-05,1.178362e-10,0.000115,0.000070
"KL(pfaX|im, pfaX|im,M)",1.121748e-06,7.353973e-11,4.021975e-05,1.889067e-06,0.000208,1.745872e-05,1.095969e-07,1.024936e-05,0.000244,0.000092
"KL(pfaZ|im, pfaZ|im,M)",3.800457e-06,3.857482e-08,5.188469e-05,7.458397e-06,0.000251,2.234066e-05,1.534661e-12,1.348286e-05,0.000164,0.000058
"KL(sdrX|im, sdrX|im,R)",1.984362e-05,1.974274e-05,5.089822e-05,9.404677e-06,0.000191,2.991830e-05,2.551409e-05,3.089973e-04,0.000620,0.000604
"KL(sdrZ|im, sdrZ|im,R)",6.819050e-06,5.156923e-06,2.470192e-05,2.049344e-06,0.000136,1.570480e-05,9.424464e-06,2.522267e-04,0.000473,0.000475
"KL(pfaX|im, pfaX|im,R)",3.065204e-08,4.047093e-08,6.181620e-05,5.896564e-08,0.000273,3.233062e-05,6.729230e-08,1.143561e-03,0.000894,0.001206
"KL(pfaZ|im, pfaZ|im,R)",1.130078e-06,1.872166e-08,9.203981e-05,7.733925e-12,0.000310,6.582995e-05,3.342123e-09,1.333758e-03,0.000775,0.000999
"KL(sdrX|im, sdrX|im,M,R)",6.541142e-05,9.527525e-05,6.716064e-05,3.887979e-05,0.000205,4.384036e-05,1.251944e-04,4.379264e-04,0.001277,0.001189
"KL(sdrZ|im, sdrZ|im,M,R)",3.336327e-05,5.150302e-05,2.867348e-05,1.629968e-05,0.000138,2.182784e-05,8.124535e-05,3.512763e-04,0.001139,0.001038


In [117]:
a = summarize_kld_results(BuildingList, baseDir, IM_list, gminfo_rotD50, pairingID = 1,
                          numGM = 826, remove_collapse = False, KL_on_residual=False,
                         use_predicted_IM=False, mv_causal_params=False)
a.style.highlight_min(color = 'lightgreen', axis = 1)

IM,SaT1,PGA,PGV,Sa_avg,CAV,SI,ASI,DSI,DS_5to75,DS_5to95
s1_48x32_high,0.000001,0.000019,0.000996,0.000103,0.002796,0.000684,0.000008,0.006085,0.003309,0.004403
s1_48x32_veryhigh,0.000001,0.000016,0.000771,0.000088,0.002197,0.000521,0.000007,0.004795,0.002698,0.003579
s1_96x48_high,0.000015,0.000030,0.001205,0.000132,0.003305,0.000832,0.000019,0.007144,0.003807,0.005106
s1_96x48_veryhigh,0.000019,0.000032,0.001149,0.000135,0.003164,0.000788,0.000023,0.006878,0.003605,0.004890
s2_48x32_high,0.000085,0.000043,0.000869,0.000140,0.002495,0.000603,0.000061,0.005547,0.002919,0.003992
s2_48x32_veryhigh,0.000120,0.000078,0.001144,0.000171,0.003086,0.000832,0.000100,0.006307,0.003431,0.004508
s2_96x48_high,0.000142,0.000066,0.000747,0.000119,0.002256,0.000534,0.000097,0.004144,0.003006,0.003664
s2_96x48_veryhigh,0.000064,0.000031,0.000710,0.000065,0.002265,0.000500,0.000039,0.004406,0.003318,0.004005
s4_96x48_high,0.000132,0.000100,0.000418,0.000076,0.001662,0.000265,0.000146,0.003124,0.003455,0.003616
s4_96x48_veryhigh,0.000036,0.000040,0.000322,0.000024,0.001374,0.000184,0.000059,0.003062,0.003377,0.003566


In [118]:
a = summarize_kld_results(BuildingList, baseDir, IM_list, gminfo_rotD50, pairingID = 1,
                          numGM = 826, remove_collapse = False, KL_on_residual=False,
                         use_predicted_IM=False, mv_causal_params=True)
a.style.highlight_min(color = 'lightgreen', axis = 1)

IM,SaT1,PGA,PGV,Sa_avg,CAV,SI,ASI,DSI,DS_5to75,DS_5to95
s1_48x32_high,0.000001,0.000019,0.000924,0.000102,0.002732,0.000670,0.000009,0.005921,0.004300,0.005047
s1_48x32_veryhigh,0.000002,0.000016,0.000714,0.000087,0.002152,0.000510,0.000009,0.004666,0.003528,0.004128
s1_96x48_high,0.000015,0.000033,0.001117,0.000135,0.003208,0.000814,0.000023,0.006939,0.004917,0.005823
s1_96x48_veryhigh,0.000019,0.000036,0.001065,0.000139,0.003080,0.000772,0.000026,0.006675,0.004658,0.005570
s2_48x32_high,0.000086,0.000055,0.000803,0.000162,0.002416,0.000589,0.000072,0.005389,0.003849,0.004606
s2_48x32_veryhigh,0.000123,0.000085,0.001064,0.000186,0.002996,0.000812,0.000104,0.006140,0.004468,0.005174
s2_96x48_high,0.000150,0.000067,0.000702,0.000129,0.002228,0.000522,0.000101,0.004081,0.004266,0.004521
s2_96x48_veryhigh,0.000118,0.000048,0.000663,0.000070,0.002197,0.000489,0.000071,0.004414,0.004908,0.005168
s4_96x48_high,0.000203,0.000211,0.000403,0.000122,0.001633,0.000268,0.000308,0.003286,0.005664,0.005424
s4_96x48_veryhigh,0.000106,0.000147,0.000324,0.000069,0.001357,0.000201,0.000207,0.003303,0.005763,0.005564
